# Выявление и визуализация сообществ в социальной сети "Вконтакте" 
### Краткое описание предметной области. 
Под графом дружеских свзяей для пользователя или нескольких пользователей понимается подграф всей соцсети, индуцированный множеством вершин, соответствующих друзьям этого пользователя или обьединению множеств друзей этих пользователей соответственно. Сам пользователь (пользователи) может быть как включен так и исключен из этого подграфа. 

Под сообществами понимаются такие множества вершин, что внутри сообщества плотность связей большая, а между сообществами мала. 

### Постановка задачи. Описание ограничений. 
Цель проекта - реализовать выгрузку графа дружеских связей пользователя или нескольких пользователей, с использованием API Вконтакте. Реализовать разбиение сети на сообщества и визуализацию полученных результатов.

Ограничения связаны с работой с API Вконтакте (Максимум 3 запроса к API в секунду.). А так же необходимостью получить наглядную визуализацию.

### Разработка
#### 1) Выгрузка данных. 

Для получения доступа к некоторым методам API Вконтакте, было необходимо создать Standalone-приложение и получить уникальный access_token. 

После этого на python был реализован набор функций выполняющих запросы к API для получения информации о 
- Информации профиля по id. (id, имя, фамилия, фото)
- Информации о друзьях пользователя по id. (список (id, имя, фамилия, фото) для каждого из друзей.
- Информации об общих друзьях двух пользователей по id. (список (id) для каждого из друзей.

Полученная информация сохранялась в виде графа библиотеки networkx.
#### 2) Выявление сообществ

Для выявления сообществ было решено воспользоваться алгоритмом [Louvain](https://arxiv.org/pdf/0803.0476.pdf). Его идея состоит в том, чтобы оптимизировать величину называемую модулярностью, которая как раз характеризует отношение между связностью вершин внутри сообществ и между сообществами. Модулярность вычисляется следующим образом :
$$Q = \frac{1}{2m}\sum_{ij}\left[A_{ij} - \frac{k_i k_j}{2m}\right]\delta(c_i, c_j)$$, где

- $A_{ij}$ - вес ребра между $i$ и $j$
- $k_i, k_j$ - взвешенные степени вершин $i,j$
- $ m $      - сумма весов всех ребер в графе
- $c_i, c_j$ - номера сообществ к которым принадлежат $i,j$
- $\delta(x, y) = \begin{cases} 0, \text{если  } x \ne y \\ 1, \text{если  } x = y\\ \end{cases}$

Алгоритм итеративно максимизирует модулярность в две фазы. 
###### Первая фаза:
Сначала каждая вершина образует отдельное сообщество. Затем для каждой вершины $i$ вычисляется изменение модулярности $\Delta Q_j^i$ после удаления $i$ из своего сообщества и перемещения его в сообщество каждого из его соседей - $j$:

$$\Delta Q_j^i = \left[ \frac{\sum_{j,in} + k_{i,in}}{2m} - \left(\frac{\sum_{j,tot} + k_{i}}{2m} \right)^2 \right] - \left[ \frac{\sum_{j,in}}{2m} - \left(\frac{\sum_{j,tot}}{2m} \right)^2 - \left(\frac{k_{i}}{2m} \right)^2 \right]$$, где
- $\sum_{j,in}$ - сумма весов всех ребер в сообществе вершины $j$
- $\sum_{j,tot}$ - сумма весов всех ребер смежных с хотя бы одной вершиной из сообщества вершины $j$
- $k_{i,in}$ - сумма весов всех ребер между сообществами вершин $i$ и $j$
- $k_i$ - взвешенная степень вершины $i$
- $ m $  - сумма весов всех ребер в графе

Затем вершина $i$ перемещается в сообщество, для которого  $\Delta Q_j^i$ максимальна (или остается на месте если ни одно перемещение не увеличит модулярность). Этот процесс повторяется пока модулярность не перестанет увеличиваться.

###### Вторая фаза:

Во второй фазе строится новый граф, вершинами которого являются сообщества, найденные на предыдущей фазе. Весами ребер между новыми вершинами становятся суммы весов всех ребер между соотвествующими сообществами. Ребра внутри сообществ превращаются в петли. После этого итерация завершается и можно начать следующую с первой фазы.

Итерации повторяются до тех пор, пока модулярность не перестанет увеличиваться. Таким образом, можно построить иерархическую структуру сообществ.

В качестве реализации этого алгоритма была выбрана python-louvain. Она является надстройкой над библиотекой networkx, которая и была использована для хранения и работы с графами.

#### 3) Визуализация графа

Networkx позволяет визуализировать графы с помощью matplotlib, однако хотелось, чтобы визуализация была максимально наглядной и желательно интерактивной, поэтому выбор пал на jsvascript-библиотеку [D3](https://d3js.org/). Она позволяет отрисовывать графы используя force-directed layout, поддерживающие интерактивное перемещение вершин. 

Для того, чтобы JS мог считать выгруженный и предобработанный с помощью python граф с уже найденными сообществами, его необходимо было сохранить в json-формате. Далее были подобраны подходящие параметры для force-directed layout, реализовано использование миниатюр аватарок пользователей в качестве вершин графа, а также увеличение фотографии и вывод имени пользователи при наведении курсора. Внутренние ребра между сообществами были окрашены в соответствующие цвета.

#### 4) Интерфейс

Для удобства использования был реализован небольшой CLI. 

```
>>> python3 main.py -h

usage: main.py [-h] (-v VKIDS [VKIDS ...] | -j JSONPATH) [-k]

optional arguments:
  -h, --help            show this help message and exit
  -v VKIDS [VKIDS ...], --vkids VKIDS [VKIDS ...]
                        Vk id or ids to add as root nodes.(may not be used
                        with -j)
  -j JSONPATH, --jsonpath JSONPATH
                        Load and visualize graph from json.(may not be used
                        with -v)
  -k, --keeproot        If provided with -v keeps the root node in resulting
                        graph. (ignored with -j)
```

Можно передать один или несколько id после ключа -v. Тогда будет выгружен граф дружеских связей для пользователей с этими id, граф будет сохранен на диске в формате json и автоматически откроется браузер с визуализацией. По умолчанию вершины самих пользователей удаляются из графа. Если передать ключ -k , то вершины самих пользователей не будут удалены.

Пример:
```
>>> python3 main.py -v 154301983

Retrieving id154301983's profile info...
Retrieving id154301983's friends and friends of friends...
100%|██████████████████████████████████████████████████████████████████| 176/176 [01:03<00:00,  2.79it/s]
Removing id154301983 ...
serving at port 8000
Created new window in existing browser session.
127.0.0.1 - - [23/Dec/2017 20:13:53] "GET /friends_network.json HTTP/1.1" 200 -

```

Можно вместо id передать путь к уже выгруженному графу в формате json после ключа -j.

Пример:

```
>>> python3 main.py -j friends_network.json
serving at port 8000
Created new window in existing browser session.
127.0.0.1 - - [23/Dec/2017 20:27:51] "GET /friends_network.json HTTP/1.1" 200 -

```

### Заключение

В результате работы было получено программное решение, позволяющее проанализировать структуру сообществ в графе дружеских связей одного или нескольких пользователей Вконтакте, визуализируя ее в наглядном, интерактивном формате.

Среди планов по улучшению и дальнейшему развитию, хотелось бы выделить:

- Использование хранимых процедур VKscript для преодоления лимита на 3 запроса в секунду и соответственно ускорения доступа к API и работы программы в целом в сотни раз.

- Вычисление различных метрик центрльности для вершин, для выявления наиболее значимых членов сообществ.

In [21]:
from IPython.core.display import HTML
HTML(' <iframe src="index.html" width="970" height="1000" frameBorder="0"></iframe>')